Building the model described in Faizi et. al. 2018 "A model of optimal protein allocation during phototrophic growth" Biosystems, 166: 26-36.

In [9]:
import tellurium as te
import math
import numpy as np
import matplotlib.pyplot as plt

In [12]:
model Faizi
    compartment cell;
    var cix; # extracellular inorganic carbon
    var ci; # intracellular inorganic carbon
    var e; # energy units (here representing the ATP, NADH, etc pool)
    var aa; # amino acids
    #var P0; # inactivated photosynthetic machinery
    #var Pstar; # activated photosynthetic machinery
    var P; # P0+Pstart
    var T; # transporter protein for inorganic carbon
    var M; # metabolic multi-enzyme complex
    var R; # ribosome
    const I;
    
    Pm = 0.0018; # permeability, dm min-1
    a_cell = 3.85e-9; # cell surface area, dm2 cell-1
    v_cell = 2.24e-14; # L, cell-1
    N_a = 6.022e23; # Avogadro constant, mol-1
    k_cat_T = 726; # maximal import rate, min-1
    K_T = 15; # half-sat constant of T, uM
    k_cat_M = 545; # maximal metabolic rate, min-1
    K_M = 2441560; # half-sat constant of M, molecules cell-1
    Y_max = 1320; # maximal translation rate, aa min-1 molecules-1
    K_aa = 10000; # half-sat constant for aa, molecule cell-1
    K_e = 10000; # half-sat constant for e, molecule cell-1
    abs_xsec = 0.166; # effective absorption cross-section of P, m2 uE-1
    turnover_P = 1900; # maximal turnover rate of P, min-1
    k_d = 1.6e-6; # rate constant for photodamage
    K_i = 1.0e8; # product-inhibition constant for e synthesis, molecule cell-1
    dens = 1.4e10; # average density, aa cell-1
    n_R = 7358; # ribosome length, aa molecule-1
    n_T = 1681; # transporter length, aa molecule-1
    n_M = 28630; # metabolic enzyme complex length, aa molecule-1
    n_P = 95451; # photosynthetic unit length, aa molecule-1
    m_u = 45; # e needed to create one aa
    m_y = 3; # e needed for one translational elongation step
    m_phi = 8; # e produced by P cycle
    m_c = 5; # average carbon chain length of aa
    
    beta_R = 0.25;
    beta_T = 0.25;
    beta_P = 0.25;
    beta_M = 0.25;
    
       
    cix=1000;
    ci=0;
    e=0;
    aa=0;
    M=1;
    R=1;
    T=1;
    P=1;
    I=1000;
    
    
    v_t := T * k_cat_T * (cix/(K_T+cix)) * (e/(K_e+e));
    v_m := M * k_cat_M * (ci/(K_M+ci)) * (e/(K_e+e));
    gamma_R := beta_R * (Y_max / n_R) * R * (e/(K_e+e)) * (aa/(K_aa+aa));
    gamma_T := beta_T * (Y_max / n_T) * R * (e/(K_e+e)) * (aa/(K_aa+aa));
    gamma_M := beta_M * (Y_max / n_M) * R * (e/(K_e+e)) * (aa/(K_aa+aa));
    gamma_P := beta_P * (Y_max / n_P) * R * (e/(K_e+e)) * (aa/(K_aa+aa));
    k2e := turnover_P / (1+(e/K_i)^4);
    v2 := (k2e*abs_xsec*I*P)/(abs_xsec*I+k2e+mu);
    translation := (n_R*gamma_R + n_T*gamma_T + n_M*gamma_M + n_P*gamma_P);
    mu := (1/dens) * (v_t/m_c);
    
    AA_P: => aa; v_m;
    AA_C: aa => ; translation + mu*aa;
    CI_P: => ci; v_t;
    CI_C: ci => ; v_m;
    M_P: => M; gamma_M;
    M_C: M =>; mu*M;
    T_P: => T; gamma_T;
    T_C: T => ; mu*T;
    R_P: => R; gamma_R;
    R_C: R => ; mu*R;
    P_P: => P; gamma_P;
    P_C: P => ; mu*P;
    e_P: => e; m_phi*v2;
    e_C: e => ; v_t+v_m*m_u + mu*e + m_y*translation;
    
    
end

Success: Model can be accessed via variable Faizi


In [13]:
Faizi.reset()
Faizi.simulate(start = 0, end = 5.64e14, steps = 1000,selections =['time','ci','e','aa'])
Faizi.plot()

Faizi.steadyStateSelections = ['ci','e','aa']
values = Faizi.getSteadyStateValues()
selections = Faizi.steadyStateSelections
print(values)
print(selections)

[  1.71474884e+21   1.19422544e-20   5.36926998e+02]
['ci', 'e', 'aa']
